In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("../")

import gurobipy
from json import dumps, loads
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as skLogisticRegression
from sklearn.metrics import (classification_report, f1_score, precision_score, recall_score)
from tqdm import tnrange, trange
import tensorflow as tf

from mlsql.influence import InfluenceRanker
from mlsql.fixer import AutoFixer
from mlsql.manager import ModelManagerLM

from models.simple_cnn import SimpleCNN
from models.logreg import LogReg
from models.linear_comb import LinearComb
from processors.adultNoCorr import AdultNoCorrProcessor


import logging
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# male vs. female original disparity
proc = AdultNoCorrProcessor()
male_ids = (proc.x_train[:, 0] == 1).numpy()
female_ids = (proc.x_train[:, 0] == 0).numpy()
one_ids = (proc.y_train == 1).numpy()
zero_ids = (proc.y_train == 0).numpy()

male_impact = (male_ids[:] & one_ids[:]).sum() / male_ids.sum()
female_impact = (female_ids[:] & one_ids[:]).sum() / female_ids.sum()
print("Train impact:", female_impact-male_impact)

male_ids = (proc.x_query[:, 0] == 1).numpy()
female_ids = (proc.x_query[:, 0] == 0).numpy()
one_ids = (proc.y_query == 1).numpy()
zero_ids = (proc.y_query == 0).numpy()

male_impact = (male_ids[:] & one_ids[:]).sum() / male_ids.sum()
female_impact = (female_ids[:] & one_ids[:]).sum() / female_ids.sum()
print("Query impact:", female_impact-male_impact)


Train impact: -0.20434572297179915
Query impact: -0.1716584657127605


In [13]:
import time
import altair as alt
alt.data_transformers.disable_max_rows()

# @tf.function
def rank_fix(ranker, fixer, n):
    rank = ranker.predict()
    fixer.fix(rank, n)
    return rank

@tf.function
def rankit(ranker):
    rank = ranker.predict()
    return rank


@tf.function
def fixit(fixer, rank, n):
    fixer.fix(rank, n)


# @tf.function
def train(manager):
    manager.fit()

In [16]:
# Logistic Regression
import time
model = LogReg(1)
manager0 = ModelManagerLM(proc.x_train, proc.y_train, model, 256)
start = time.time()
manager0.fit(print_value=True, max_iter=2000, tol=1e-5)
print(time.time() - start)
manager0.report(proc.x_train, proc.y_train, proc.x_test, proc.y_test)

SGD loss: tf.Tensor(0.3833263, shape=(), dtype=float32)
SGD steps: 1040
9.171545505523682
Model name: LogReg
On Training
               precision    recall  f1-score   support

         0.0       0.85      0.92      0.88     16672
         1.0       0.66      0.48      0.56      5306

    accuracy                           0.82     21978
   macro avg       0.75      0.70      0.72     21978
weighted avg       0.80      0.82      0.80     21978

On Testing
               precision    recall  f1-score   support

         0.0       0.86      0.93      0.89      1867
         1.0       0.68      0.50      0.58       575

    accuracy                           0.83      2442
   macro avg       0.77      0.71      0.73      2442
weighted avg       0.82      0.83      0.82      2442



In [17]:
# Rain Debug
from tqdm.notebook import tnrange, trange
K = 5000
corrsel = proc.corrsel
manager = ModelManagerLM(proc.x_train, proc.y_train, LogReg(1), 256)
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
ranker = InfluenceRanker(manager=manager, on=proc.complain)
fixer = AutoFixer(manager, corrsel, K)

impacts = []
AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_rain = 0
AQ = proc.complain(manager).AQ
f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
AQs.append(float(AQ))
weighted_f1.append(f1)

x, y = manager.get_remaining()
male_ids = (x[:, 0] == 1).numpy()
female_ids = (x[:, 0] == 0).numpy()
one_ids = (y == 1).numpy()
zero_ids = (y == 0).numpy()
male_impact = (male_ids[:] & one_ids[:]).sum() / male_ids.sum()
female_impact = (female_ids[:] & one_ids[:]).sum() / female_ids.sum()
impacts.append(abs(female_impact - male_impact))

step_size = 100
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
    rank = rank_fix(ranker, fixer, nfix)
    middle = time.time()
    manager.fit(max_iter=1000, tol=1e-5, print_value=True)
    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += end - middle
    
    x, y = manager.get_remaining()
    male_ids = (x[:, 0] == 1).numpy()
    female_ids = (x[:, 0] == 0).numpy()
    one_ids = (y == 1).numpy()
    zero_ids = (y == 0).numpy()

    male_impact = (male_ids[:] & one_ids[:]).sum() / male_ids.sum()
    female_impact = (female_ids[:] & one_ids[:]).sum() / female_ids.sum()
    impact = abs(female_impact - male_impact)
    AQ = proc.complain(manager).AQ
    f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
    impacts.append(impact)
    AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_rain)
print("Model_time:", model_time_rain)

df_rain = pd.DataFrame({
    "Complain": np.array(AQs),
    "Impact": np.array(impacts),
    "F1": np.array(weighted_f1),
    "K": [1] + list(range(step_size, K + step_size, step_size)),
    "Method": np.repeat("Rain", len(AQs)),
})
alt.Chart(pd.concat([df_rain])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain.shape[0], grid=False)),
    alt.Y("Complain:Q", scale=alt.Scale(domain=[min(AQs),max(AQs)])),
    color = "Method"
)

  0%|          | 0/50 [00:00<?, ?it/s]

SGD loss: tf.Tensor(0.37715793, shape=(), dtype=float32)
SGD steps: 106
SGD loss: tf.Tensor(0.36835492, shape=(), dtype=float32)
SGD steps: 130
SGD loss: tf.Tensor(0.35833958, shape=(), dtype=float32)
SGD steps: 168
SGD loss: tf.Tensor(0.34994972, shape=(), dtype=float32)
SGD steps: 153
SGD loss: tf.Tensor(0.34297037, shape=(), dtype=float32)
SGD steps: 172
SGD loss: tf.Tensor(0.33488727, shape=(), dtype=float32)
SGD steps: 208
SGD loss: tf.Tensor(0.32790536, shape=(), dtype=float32)
SGD steps: 175
SGD loss: tf.Tensor(0.32162884, shape=(), dtype=float32)
SGD steps: 191
SGD loss: tf.Tensor(0.31499204, shape=(), dtype=float32)
SGD steps: 184
SGD loss: tf.Tensor(0.31022674, shape=(), dtype=float32)
SGD steps: 96
SGD loss: tf.Tensor(0.3064186, shape=(), dtype=float32)
SGD steps: 103
SGD loss: tf.Tensor(0.30335552, shape=(), dtype=float32)
SGD steps: 108
SGD loss: tf.Tensor(0.2990878, shape=(), dtype=float32)
SGD steps: 100
SGD loss: tf.Tensor(0.2957972, shape=(), dtype=float32)
SGD steps: 

alt.Chart(...)

In [12]:
alt.Chart(pd.concat([df_rain])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain.shape[0], grid=False)),
    alt.Y("Impact:Q"),
    color = "Method"
)

alt.Chart(...)

In [13]:
alt.Chart(pd.concat([df_rain])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain.shape[0], grid=False)),
    alt.Y("F1:Q", scale=alt.Scale(domain=[0,1])),
    color = "Method"
)

alt.Chart(...)

In [14]:
# Frog Model
from models.linear_comb_test import LinearCombTest
from mlsql.manager_test import ModelManagerTest

model = LinearCombTest(1)
manager_test0 = ModelManagerTest(proc.x_a_train, proc.x_b_train, proc.y_train, model, 256)
start = time.time()
# tol=1e-7, lr=0.5
manager_test0.fit(print_value=True, tol=1e-10, lr=0.02, max_iter=50000)
print(time.time() - start)
manager_test0.report(proc.x_a_train, proc.x_b_train, proc.y_train, proc.x_a_test, proc.x_b_test, proc.y_test)
manager_test0.report(proc.x_a_train, proc.x_b_train, proc.y_train, proc.x_a_query, proc.x_b_query, proc.y_query)

SGD loss: tf.Tensor(0.06862382, shape=(), dtype=float32)
SGD steps: 49999
591.2330694198608
Model name: LinearCombTest
On Training
               precision    recall  f1-score   support

         0.0       0.80      0.97      0.88     16672
         1.0       0.74      0.23      0.35      5306

    accuracy                           0.79     21978
   macro avg       0.77      0.60      0.61     21978
weighted avg       0.78      0.79      0.75     21978

On Testing
               precision    recall  f1-score   support

         0.0       0.81      0.98      0.89      1867
         1.0       0.77      0.26      0.39       575

    accuracy                           0.81      2442
   macro avg       0.79      0.62      0.64      2442
weighted avg       0.80      0.81      0.77      2442

Model name: LinearCombTest
On Training
               precision    recall  f1-score   support

         0.0       0.80      0.97      0.88     16672
         1.0       0.74      0.23      0.35      5306

In [ ]:
AQ = proc.test_complain(manager_test).AQ
f1 = f1_score(proc.y_test.numpy(), manager_test.model.predict(proc.x_a_test, proc.x_b_test).numpy(), average='weighted')
# f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
x_a, _, y = manager_test.get_remaining()
male_ids = (x_a[:, 0] == 1).numpy()
female_ids = (x_a[:, 0] == 0).numpy()
one_ids = (y == 1).numpy()
zero_ids = (y == 0).numpy()

male_impact = (male_ids[:] & one_ids[:]).sum() / male_ids.sum()
female_impact = (female_ids[:] & one_ids[:]).sum() / female_ids.sum()
AQs.append(float(AQ))
weighted_f1.append(f1)
impacts.append(abs(female_impact - male_impact))
print(male_impact, female_impact)

In [15]:
#Frog Debug
K=5000
manager_test = ModelManagerTest(proc.x_a_train, proc.x_b_train, proc.y_train, LinearCombTest(1), 256)
manager_test.model.set_weights(manager_test0.model.get_weights())
manager_test.delta = tf.Variable(manager_test0.delta.value(), name="delta")
ranker = InfluenceRanker(manager=manager_test, on=proc.test_complain)
fixer = AutoFixer(manager_test, proc.corrsel, K)

impacts = []
test_impacts = []
AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_rain = 0

AQ = proc.test_complain(manager_test).AQ
f1 = f1_score(proc.y_test.numpy(), manager_test.model.predict(proc.x_a_test, proc.x_b_test).numpy(), average='weighted')
# f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
x_a, _, y = manager_test.get_remaining()
male_ids = (x_a[:, 0] == 1).numpy()
female_ids = (x_a[:, 0] == 0).numpy()
one_ids = (y == 1).numpy()
zero_ids = (y == 0).numpy()

male_impact = (male_ids[:] & one_ids[:]).sum() / male_ids.sum()
female_impact = (female_ids[:] & one_ids[:]).sum() / female_ids.sum()
AQs.append(float(AQ))
weighted_f1.append(f1)
impacts.append(abs(female_impact - male_impact))

y = tf.squeeze(manager_test.model(proc.x_a_test, proc.x_b_test))
male_ids = (proc.x_a_test[:, 0] == 1).numpy()
female_ids = (proc.x_a_test[:, 0] == 0).numpy()
one_ids = (y >= 0.5).numpy()
zero_ids = (y < 0.5).numpy()

male_impact = (male_ids[:] & one_ids[:]).sum() / male_ids.sum()
female_impact = (female_ids[:] & one_ids[:]).sum() / female_ids.sum()
test_impacts.append(abs(female_impact / male_impact))

step_size = 100
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
    rank = rank_fix(ranker, fixer, nfix)
    middle = time.time()
    # tol=1e-7, lr=0.5
    manager_test.fit(max_iter=10000, tol=1e-7, lr=0.02, print_value=True)
    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += end - middle
    
    x_a, _, y = manager_test.get_remaining()
    male_ids = (x_a[:, 0] == 1).numpy()
    female_ids = (x_a[:, 0] == 0).numpy()
    one_ids = (y == 1).numpy()
    zero_ids = (y == 0).numpy()

    male_impact = (male_ids[:] & one_ids[:]).sum() / male_ids.sum()
    female_impact = (female_ids[:] & one_ids[:]).sum() / female_ids.sum()
    impact = abs(female_impact - male_impact)

    AQ = proc.test_complain(manager_test).AQ
    f1 = f1_score(proc.y_test.numpy(), manager_test.model.predict(proc.x_a_test, proc.x_b_test).numpy(), average='weighted')
#     f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
    impacts.append(impact)
    AQs.append(float(AQ))
    weighted_f1.append(f1)
    
    y = tf.squeeze(manager_test.model(proc.x_a_test, proc.x_b_test))
    male_ids = (proc.x_a_test[:, 0] == 1).numpy()
    female_ids = (proc.x_a_test[:, 0] == 0).numpy()
    one_ids = (y >= 0.5).numpy()
    zero_ids = (y < 0.5).numpy()

    male_impact = (male_ids[:] & one_ids[:]).sum() / male_ids.sum()
    female_impact = (female_ids[:] & one_ids[:]).sum() / female_ids.sum()
    test_impacts.append(abs(female_impact - male_impact))
    

print("Rank_time:", rank_time_rain)
print("Model_time:", model_time_rain)
AC = proc.test_complain(manager_test).AC

df_rain_test = pd.DataFrame({
    "Complain": np.array(AQs) - AC,
    "Impact": np.array(impacts),
    "F1": np.array(weighted_f1),
    "K": list(range(0, K, step_size)) + [K],
    "Method": np.repeat("Rain", len(AQs)),
    "Test_impact": np.array(test_impacts),
})
alt.Chart(pd.concat([df_rain_test])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain_test.shape[0], grid=False)),
    alt.Y("Complain:Q"),
    color = "Method"
)

  2%|▏         | 1/50 [00:01<01:28,  1.80s/it]

SGD loss: tf.Tensor(0.06791393, shape=(), dtype=float32)
SGD steps: 1


  4%|▍         | 2/50 [00:02<00:55,  1.17s/it]

SGD loss: tf.Tensor(0.066932335, shape=(), dtype=float32)
SGD steps: 1


  6%|▌         | 3/50 [00:03<00:47,  1.01s/it]

...

 98%|█████████▊| 49/50 [11:08<00:17, 17.43s/it]

SGD loss: tf.Tensor(0.03171212, shape=(), dtype=float32)
SGD steps: 985


100%|██████████| 50/50 [11:27<00:00, 13.75s/it]

SGD loss: tf.Tensor(0.030328289, shape=(), dtype=float32)
SGD steps: 1604
Rank_time: 29.96768307685852
Model_time: 656.5517866611481


alt.Chart(...)

In [24]:
impacts

[0.20434572297179915,
 0.20211052822818731,
 0.1988093242041537,
 0.1947221542128577,
 0.19075926422447045,
 0.1873639670956407,
 0.18336240444877244,
 0.1792521114835105,
 0.17534330824415323,
 0.1714321859564591,
 0.16836575240515972,
 0.1652004774158927,
 0.16225240756417791,
 0.15835751679125173,
 0.15481830305129807,
 0.15006047836126046,
 0.1448407158082764,
 0.13939758158664695,
 0.13425823112472482,
 0.1290517883239138,
 0.1237371220987879,
 0.11879612574508847,
 0.1136742753566663,
 0.10993929751732054,
 0.1058794192806545,
 0.10252840264491744,
 0.09722298328927476,
 0.09359156508729578,
 0.08779529664986536,
 0.08165391705611673,
 0.07604430681490854,
 0.0722545596713136,
 0.06739181179961354,
 0.0620665228234024,
 0.07640260604632199,
 0.09121489200044633,
 0.10652750850529268,
 0.12236624136488378,
 0.11738051025351204,
 0.11081825480354059,
 0.10403568049889668,
 0.09714088623553925,
 0.11361678098164718,
 0.10660695845230117,
 0.09947920485752201,
 0.09223051889771436,
 

In [21]:
weighted_f1

[0.8174107060320042,
 0.8186911243630448,
 0.8186911243630448,
 0.817829799538166,
 0.8191974383083428,
 0.8213647503031832,
 0.8215729011228387,
 0.8228490894228463,
 0.822641296626279,
 0.8217206063838446,
 0.8220766195721473,
 0.8219288738400308,
 0.8212170863333655,
 0.8244251801061719,
 0.8223982864724559,
 0.8223432919793787,
 0.8194194162443214,
 0.8195786766372979,
 0.8195786766372979,
 0.8190266568351066,
 0.8142659377271675,
 0.8142544678848405,
 0.8130256629526145,
 0.8130256629526145,
 0.8130256629526145,
 0.8112795279717178,
 0.8114562903513394,
 0.8109268616368009,
 0.8112786520037542,
 0.807531479000764,
 0.8063552787122423,
 0.8054608458194581,
 0.8037143135512194,
 0.8038587303612503,
 0.8033791341113315,
 0.8026100293473912,
 0.8049130049505382,
 0.8044886860313858,
 0.802014344005774,
 0.8014472235501648,
 0.7928569724189309,
 0.7921075756864786,
 0.792627663499755,
 0.7915948149584267,
 0.7893187974368786,
 0.7887077053905077,
 0.7890479892093621,
 0.789047989209362